# Kernel imp 2

In [1]:
import numpy as np
from scipy.spatial.distance import cdist
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from numpy import linalg as LA
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# hyperparameters: Kernel, Regularization, Gamma

In [23]:
# load new data
bankdata = pd.read_csv("Dataset/bill_authentication.csv")
# bankdata.shape

X = bankdata.drop('Class', axis=1)
y = bankdata['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# print(X_test)

In [6]:
svclassifier = SVC(kernel="linear")
svclassifier.fit(X_train, y_train)

In [12]:
print("test size", len(X_test))
y_pred = svclassifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))



test size 275
[[145   0]
 [  0 130]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       145
           1       1.00      1.00      1.00       130

   micro avg       1.00      1.00      1.00       275
   macro avg       1.00      1.00      1.00       275
weighted avg       1.00      1.00      1.00       275



In [92]:
# load iris
iris = load_iris()
X = iris.data[:,:2]
y = iris.target

X = iris.data[y != 0]
y = iris.target[y != 0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9)

print(len(y_train))
svc2 = SVC(kernel="linear", gamma="scale")
svc2.fit(X_train, y_train)

y_pred = svc2.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

10
[[42  2]
 [ 0 46]]
              precision    recall  f1-score   support

           1       1.00      0.95      0.98        44
           2       0.96      1.00      0.98        46

   micro avg       0.98      0.98      0.98        90
   macro avg       0.98      0.98      0.98        90
weighted avg       0.98      0.98      0.98        90



In [93]:
n_samples_train = len(X_train)
# print(n_samples_train)
X_K = np.concatenate((X_train, X_test))

K = rbf_kernel(X_K, 1)
D = make_D_matrix(K)
L = make_L_matrix(K, D)

L, D, K = apply_transfer_func(L, D, K)

K_train = K[:n_samples_train,:n_samples_train]
K_test = K[n_samples_train:, :n_samples_train]

svc_k = SVC(kernel="precomputed", gamma="scale")
svc_k.fit(K_train, y_train)

y_pred_k = svc_k.predict(K_test)

print(confusion_matrix(y_test, y_pred_k))
print(classification_report(y_test, y_pred_k))


[[43  1]
 [11 35]]
              precision    recall  f1-score   support

           1       0.80      0.98      0.88        44
           2       0.97      0.76      0.85        46

   micro avg       0.87      0.87      0.87        90
   macro avg       0.88      0.87      0.87        90
weighted avg       0.89      0.87      0.87        90



In [32]:
X_K.shape

(100, 4)

In [ ]:









    
    # Make Kernel
    K = rbf_kernel(X, 1)
    D = make_D_matrix(K)
    L = make_L_matrix(K, D)
    
    L, D, K = apply_transfer_func(L, D, K)
    
    # Remove data without labels
    K_red = K[:70,:70]
    Y_red = Y[:70]
    
    K_test = K[70:100,:70]
    
    print(K_red.shape)
    
    # Apply to SVM
    clf = svm.SVC(kernel="precomputed", C=1)
    clf.fit(K_red, Y_red)
    
    # Predict and accuracy
    y_pred = clf.predict(K_test)
    
    #print('t', Y[70:100])
    #print('y', y_pred)
    print("accuracy:", accuracy(y_pred, Y[70:100]))
    
    
    # Basic SVM
    clf2 = svm.SVC(kernel="linear", C=1)
    clf2.fit(X[:70], Y_red)
    
    y_pred2 = clf2.predict(X[70:100])
    print("accuracy:", accuracy(y_pred2, Y[70:100]))


## 2.3 kernel induced by a clustered representation

In [70]:
def rbf_kernel(X, sigma, diag=1):
    N = X.shape[0]
    K = np.zeros((N,N))

    for i in range(N):
        for j in range(N):
            if i == j:
                K[i,j] = diag
            else:
                x_i = X[i, :].reshape(1,-1)
                x_j = X[j, :].reshape(1,-1)

                K[i, j] = np.exp(-cdist(x_i, x_j, 'sqeuclidean') / (sigma ** 2)) # euclidean?
    return K


def make_D_matrix(K):
    K_sum = np.sum(K, axis=1)
    D = np.diag(K_sum)

    return D


def make_L_matrix(K, D):
    D_temp = np.diag( np.diag(D) ** -0.5 )
    L = D_temp @ K @ D_temp
    
    w, v = LA.eig(L) # w = eigenvalues, v = normalized (unit “length”) eigenvectors
    
    return L

def step_transfer(L, lambda_cut):
    w, v = LA.eig(L) # w = eigenvalues
    return np.where(w > lambda_cut, 1, 0)

def apply_transfer_func(L, D, K, type="linear"):
    if type == "linear":
        return L, D, K
    if type == "step":
        None
        
    L_hat = 0
    D_hat = 0
    K_hat = 0
    
    return L_hat, D_hat, K_hat


def test_run(X):
    X = X[:3, :] # "testing with a small X"
    
    K = rbf_kernel(X, 1)
    D = make_D_matrix(K)
    L = make_L_matrix(K, D)
    
    L, D, K = apply_transfer_func(L, D, K)
    
    # print(K, "\n\n", D, "\n\n", L)    



# test_run(X)   

In [11]:
# Basic SVM with predefined kernel matrix

def accuracy(t, y):
    val = 0.0
    N = len(t)
    for i in range(N):
        if t[i] == y[i]:
            val += 1
    
    return val / N


def test_svm(X, Y):
    # Shuffle data
    np.random.seed(40)
    n_sample = len(X)
    order = np.random.permutation(n_sample)
    X = X[order]
    Y = Y[order].astype(np.float)
    
    # Make Kernel
    K = rbf_kernel(X, 1)
    D = make_D_matrix(K)
    L = make_L_matrix(K, D)
    
    L, D, K = apply_transfer_func(L, D, K)
    
    # Remove data without labels
    K_red = K[:70,:70]
    Y_red = Y[:70]
    
    K_test = K[70:100,:70]
    
    print(K_red.shape)
    
    # Apply to SVM
    clf = svm.SVC(kernel="precomputed", C=1)
    clf.fit(K_red, Y_red)
    
    # Predict and accuracy
    y_pred = clf.predict(K_test)
    
    #print('t', Y[70:100])
    #print('y', y_pred)
    print("accuracy:", accuracy(y_pred, Y[70:100]))
    
    
    # Basic SVM
    clf2 = svm.SVC(kernel="linear", C=1)
    clf2.fit(X[:70], Y_red)
    
    y_pred2 = clf2.predict(X[70:100])
    print("accuracy:", accuracy(y_pred2, Y[70:100]))

    
test_svm(X, Y)


(70, 70)
accuracy: 1.0
accuracy: 1.0
